In [1]:
import sys
sys.path.append(sys.path[0] + '/bert')
import modeling
import optimization
import tokenization
import run_classifier
import tensorflow as tf
import tensorflow.contrib.slim as slim


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [11]:
tf.logging.set_verbosity(tf.logging.DEBUG)
flags = {
    'do_lower_case': True,
    'bert_config_file': '/data0/ultraman/uncased_L-12_H-768_A-12/bert_config.json',
    'init_checkpoint': '/data0/ultraman/uncased_L-12_H-768_A-12/bert_model.ckpt',
    'vocab_file': '/data0/ultraman/uncased_L-12_H-768_A-12/vocab.txt',
    'use_tpu': False,
    'data_dir': '/data0/ultraman/datasets/MNLI',
    'task_name': 'MNLI',
    'learning_rate': 5e-5,
    'is_training': False
}
tokenization.validate_case_matches_checkpoint(flags['do_lower_case'],
flags['init_checkpoint'])
bert_config = modeling.BertConfig.from_json_file(flags['bert_config_file'])
processors = {
      "cola": run_classifier.ColaProcessor,
      "mnli": run_classifier.MnliProcessor,
      "mrpc": run_classifier.MrpcProcessor,
      "xnli": run_classifier.XnliProcessor,
      "mnli": run_classifier.MnliProcessor,
      "qnli": run_classifier.QnliProcessor,
      "qqp": run_classifier.QqpProcessor,
      "rte": run_classifier.RteProcessor,
      "wnli": run_classifier.WnliProcessor,
      "sst-2": run_classifier.Sst2Processor,
      "mrpc": run_classifier.MrpcProcessor,
      "sts-b": run_classifier.StsProcessor,
  }

In [12]:
tokenizer = tokenization.FullTokenizer(vocab_file=flags['vocab_file'], do_lower_case=flags['do_lower_case'])
task_name = flags['task_name'].lower()
processor = processors[task_name]()
label_list = processor.get_labels()
num_train_steps = None
num_warmup_steps = None
input_ids = tf.constant([[31, 51, 99], [15, 5, 0]])
model = modeling.BertModel(
    config=bert_config,
    is_training=flags['is_training'],
    input_ids=input_ids)

In [13]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

orm/gamma:0 (float32_ref 768) [768, bytes: 3072]
bert_1/encoder/layer_11/attention/self/query/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert_1/encoder/layer_11/attention/self/query/bias:0 (float32_ref 768) [768, bytes: 3072]
bert_1/encoder/layer_11/attention/self/key/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert_1/encoder/layer_11/attention/self/key/bias:0 (float32_ref 768) [768, bytes: 3072]
bert_1/encoder/layer_11/attention/self/value/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert_1/encoder/layer_11/attention/self/value/bias:0 (float32_ref 768) [768, bytes: 3072]
bert_1/encoder/layer_11/attention/output/dense/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert_1/encoder/layer_11/attention/output/dense/bias:0 (float32_ref 768) [768, bytes: 3072]
bert_1/encoder/layer_11/attention/output/LayerNorm/beta:0 (float32_ref 768) [768, bytes: 3072]
bert_1/encoder/layer_11/attention/output/LayerNorm/gamma:0 (float32_ref 768) [768, bytes: 3072]


In [16]:
sess = tf.Session()
 # 保存路径
tenboard_dir = './tensorboard/test2/123'

# 指定一个文件用来保存图
writer = tf.summary.FileWriter(tenboard_dir)
# 把图add进去
writer.add_graph(sess.graph)


In [28]:
import os
from tensorflow.python import pywrap_tensorflow

checkpoint_path = os.path.join("/data0/ultraman/uncased_L-12_H-768_A-12/", "bert_model.ckpt")
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path) #tf.train.NewCheckpointReader
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print("tensor_name: ", key)
    if key == 'bert/encoder/layer_0/attention/self/query/kernel':
        print(reader.get_tensor(key).shape)

tensor_name:  cls/seq_relationship/output_bias
tensor_name:  cls/predictions/transform/dense/kernel
tensor_name:  cls/predictions/transform/dense/bias
tensor_name:  cls/predictions/output_bias
tensor_name:  bert/encoder/layer_3/intermediate/dense/kernel
tensor_name:  bert/encoder/layer_3/attention/self/value/kernel
tensor_name:  bert/encoder/layer_0/attention/self/key/kernel
tensor_name:  bert/encoder/layer_11/attention/output/LayerNorm/beta
tensor_name:  bert/embeddings/LayerNorm/beta
tensor_name:  bert/encoder/layer_2/intermediate/dense/kernel
tensor_name:  bert/encoder/layer_7/intermediate/dense/kernel
tensor_name:  bert/encoder/layer_2/attention/self/value/bias
tensor_name:  bert/encoder/layer_2/attention/self/key/bias
tensor_name:  bert/encoder/layer_2/attention/output/dense/bias
tensor_name:  bert/encoder/layer_10/attention/output/LayerNorm/gamma
tensor_name:  bert/encoder/layer_9/attention/self/value/kernel
tensor_name:  bert/encoder/layer_2/attention/output/LayerNorm/gamma
tens

In [29]:
for var in tf.trainable_variables():
    print(var.name)

dings/position_embeddings:0
bert_1/embeddings/LayerNorm/beta:0
bert_1/embeddings/LayerNorm/gamma:0
bert_1/encoder/layer_0/attention/self/query/kernel:0
bert_1/encoder/layer_0/attention/self/query/bias:0
bert_1/encoder/layer_0/attention/self/key/kernel:0
bert_1/encoder/layer_0/attention/self/key/bias:0
bert_1/encoder/layer_0/attention/self/value/kernel:0
bert_1/encoder/layer_0/attention/self/value/bias:0
bert_1/encoder/layer_0/attention/output/dense/kernel:0
bert_1/encoder/layer_0/attention/output/dense/bias:0
bert_1/encoder/layer_0/attention/output/LayerNorm/beta:0
bert_1/encoder/layer_0/attention/output/LayerNorm/gamma:0
bert_1/encoder/layer_0/intermediate/dense/kernel:0
bert_1/encoder/layer_0/intermediate/dense/bias:0
bert_1/encoder/layer_0/output/dense/kernel:0
bert_1/encoder/layer_0/output/dense/bias:0
bert_1/encoder/layer_0/output/LayerNorm/beta:0
bert_1/encoder/layer_0/output/LayerNorm/gamma:0
bert_1/encoder/layer_1/attention/self/query/kernel:0
bert_1/encoder/layer_1/attention/s

(2,)
